In [1]:
# Célula 2: Código do Modelo de Otimização Estocástica

import pulp
import random

# --------------------------------------------------------------------------
# 1. DEFINIÇÃO DOS DADOS BASE (Extraídos do Artigo - Situação I)
# --------------------------------------------------------------------------
# [cite_start]Fonte: Parente Filho, E. G. (1995), conforme citado no documento anexo[cite: 57].

num_strata = 5

# [cite_start]Coeficientes da função objetivo (receita líquida em US$/ha) [cite: 58]
base_revenue_c = [404.40, 286.5, 183.3, 102.57, 127.0]

# [cite_start]Limites de área máxima por estrato (ha) [cite: 59, 60, 61, 62, 63]
area_limits_A = [21.66, 14.17, 11.33, 6.34, 7.83]

# Coeficientes técnicos das restrições
# [cite_start]Produtividade de lenha (unid/ha) [cite: 64]
base_productivity_r = [50.57, 53.08, 43.09, 40.71, 38.69]
# [cite_start]Demanda mínima de lenha (unid) [cite: 64]
firewood_demand_min = 2300

# [cite_start]Mão de obra (h-H/ha) [cite: 65]
base_labor_m = [87.5, 68.3, 58.0, 56.5, 48.6]
# [cite_start]Disponibilidade base de mão de obra (h-H) [cite: 65]
base_labor_M_max = 3500

# --------------------------------------------------------------------------
# 2. GERAÇÃO DE CENÁRIOS ESTOCÁSTICOS
# --------------------------------------------------------------------------
# O artigo propõe tratar produtividade, preços (receita) e disponibilidade
# [cite_start]de mão de obra como incertos[cite: 325].
# Vamos gerar cenários para simular essa incerteza.

num_scenarios = 50  # Número de cenários a serem simulados
probabilities = [1.0 / num_scenarios] * num_scenarios

# Custos de Recurso (suposições, já que não estão no artigo)
q_mo = 10.0  # Custo por hora de mão de obra extra (maior que o implícito na receita)
q_deficit = 50.0 # Penalidade por unidade de lenha não entregue

scenarios = []
print(f"Gerando {num_scenarios} cenários de incerteza...")
for i in range(num_scenarios):
    # Fator de variabilidade (ex: +/- 15% em torno da base)
    variability_factor = random.uniform(0.85, 1.15)

    # Gera parâmetros incertos para este cenário
    scenario_c = [c * random.uniform(0.85, 1.15) for c in base_revenue_c]
    scenario_r = [r * random.uniform(0.85, 1.15) for r in base_productivity_r]
    scenario_M_max = base_labor_M_max * random.uniform(0.90, 1.10)

    scenarios.append({
        "c": scenario_c,
        "r_lenha": scenario_r,
        "M_max": scenario_M_max
    })
print("Cenários gerados com sucesso.\n")

# --------------------------------------------------------------------------
# 3. MODELAGEM COM PULP
# --------------------------------------------------------------------------

# Inicializa o modelo
model = pulp.LpProblem("GestaoFlorestalEstocastica", pulp.LpMaximize)

# --- Variáveis de Decisão ---

# Variáveis de Primeiro Estágio (não dependem do cenário)
E = [pulp.LpVariable(f'E_{j+1}', lowBound=0, cat='Continuous') for j in range(num_strata)]

# Variáveis de Segundo Estágio (dependem do cenário)
y_mo = pulp.LpVariable.dicts("y_mo", range(num_scenarios), lowBound=0, cat='Continuous')
y_deficit = pulp.LpVariable.dicts("y_deficit", range(num_scenarios), lowBound=0, cat='Continuous')

# --- Função Objetivo ---
# [cite_start]Maximizar a receita esperada menos o custo de recurso esperado [cite: 327]
expected_revenue = pulp.lpSum(
    probabilities[s] * pulp.lpSum(scenarios[s]['c'][j] * E[j] for j in range(num_strata))
    for s in range(num_scenarios)
)

expected_recourse_cost = pulp.lpSum(
    probabilities[s] * (q_mo * y_mo[s] + q_deficit * y_deficit[s])
    for s in range(num_scenarios)
)

model += expected_revenue - expected_recourse_cost, "Lucro_Esperado_Liquido"

# --- Restrições ---

# Restrições de Primeiro Estágio
for j in range(num_strata):
    model += E[j] <= area_limits_A[j], f'Limite_Area_E{j+1}'

# Restrições de Segundo Estágio (para cada cenário)
for s in range(num_scenarios):
    # Restrição de Mão de Obra
    labor_needed = pulp.lpSum(base_labor_m[j] * E[j] for j in range(num_strata))
    model += labor_needed <= scenarios[s]['M_max'] + y_mo[s], f'Restricao_MO_Cenario_{s+1}'

    # Restrição de Demanda de Lenha
    firewood_produced = pulp.lpSum(scenarios[s]['r_lenha'][j] * E[j] for j in range(num_strata))
    model += firewood_produced >= firewood_demand_min - y_deficit[s], f'Restricao_Lenha_Cenario_{s+1}'

# --------------------------------------------------------------------------
# 4. RESOLUÇÃO E APRESENTAÇÃO DOS RESULTADOS
# --------------------------------------------------------------------------

# Resolve o problema
print("Resolvendo o modelo de otimização estocástica...")
model.solve()
print("Modelo resolvido.\n")

# Exibe os resultados
print(f"Status da Solução: {pulp.LpStatus[model.status]}\n")

print("--- Decisões Ótimas de Alocação de Área (1º Estágio) ---")
print("Esta é a estratégia robusta a ser implementada, considerando as incertezas:")
for j in range(num_strata):
    print(f"Área a ser explorada no Estrato E_{j+1}: {E[j].varValue:.4f} ha")

print("\n--- Valor Esperado da Função Objetivo ---")
print(f"Lucro Líquido Esperado: US$ {pulp.value(model.objective):.2f}\n")

print("--- Análise dos Custos de Recurso Esperados ---")
total_expected_mo_cost = sum(probabilities[s] * q_mo * y_mo[s].varValue for s in range(num_scenarios))
total_expected_deficit_cost = sum(probabilities[s] * q_deficit * y_deficit[s].varValue for s in range(num_scenarios))

print(f"Custo esperado com contratação de mão de obra extra: US$ {total_expected_mo_cost:.2f}")
print(f"Custo esperado com penalidades por déficit de lenha: US$ {total_expected_deficit_cost:.2f}")

ModuleNotFoundError: No module named 'pulp'

In [2]:
%pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 84.2 MB/s eta 0:00:00


In [3]:
# Célula 2: Código do Modelo de Otimização Estocástica

import pulp
import random

# --------------------------------------------------------------------------
# 1. DEFINIÇÃO DOS DADOS BASE (Extraídos do Artigo - Situação I)
# --------------------------------------------------------------------------
# [cite_start]Fonte: Parente Filho, E. G. (1995), conforme citado no documento anexo[cite: 57].

num_strata = 5

# [cite_start]Coeficientes da função objetivo (receita líquida em US$/ha) [cite: 58]
base_revenue_c = [404.40, 286.5, 183.3, 102.57, 127.0]

# [cite_start]Limites de área máxima por estrato (ha) [cite: 59, 60, 61, 62, 63]
area_limits_A = [21.66, 14.17, 11.33, 6.34, 7.83]

# Coeficientes técnicos das restrições
# [cite_start]Produtividade de lenha (unid/ha) [cite: 64]
base_productivity_r = [50.57, 53.08, 43.09, 40.71, 38.69]
# [cite_start]Demanda mínima de lenha (unid) [cite: 64]
firewood_demand_min = 2300

# [cite_start]Mão de obra (h-H/ha) [cite: 65]
base_labor_m = [87.5, 68.3, 58.0, 56.5, 48.6]
# [cite_start]Disponibilidade base de mão de obra (h-H) [cite: 65]
base_labor_M_max = 3500

# --------------------------------------------------------------------------
# 2. GERAÇÃO DE CENÁRIOS ESTOCÁSTICOS
# --------------------------------------------------------------------------
# O artigo propõe tratar produtividade, preços (receita) e disponibilidade
# [cite_start]de mão de obra como incertos[cite: 325].
# Vamos gerar cenários para simular essa incerteza.

num_scenarios = 50  # Número de cenários a serem simulados
probabilities = [1.0 / num_scenarios] * num_scenarios

# Custos de Recurso (suposições, já que não estão no artigo)
q_mo = 10.0  # Custo por hora de mão de obra extra (maior que o implícito na receita)
q_deficit = 50.0 # Penalidade por unidade de lenha não entregue

scenarios = []
print(f"Gerando {num_scenarios} cenários de incerteza...")
for i in range(num_scenarios):
    # Fator de variabilidade (ex: +/- 15% em torno da base)
    variability_factor = random.uniform(0.85, 1.15)

    # Gera parâmetros incertos para este cenário
    scenario_c = [c * random.uniform(0.85, 1.15) for c in base_revenue_c]
    scenario_r = [r * random.uniform(0.85, 1.15) for r in base_productivity_r]
    scenario_M_max = base_labor_M_max * random.uniform(0.90, 1.10)

    scenarios.append({
        "c": scenario_c,
        "r_lenha": scenario_r,
        "M_max": scenario_M_max
    })
print("Cenários gerados com sucesso.\n")

# --------------------------------------------------------------------------
# 3. MODELAGEM COM PULP
# --------------------------------------------------------------------------

# Inicializa o modelo
model = pulp.LpProblem("GestaoFlorestalEstocastica", pulp.LpMaximize)

# --- Variáveis de Decisão ---

# Variáveis de Primeiro Estágio (não dependem do cenário)
E = [pulp.LpVariable(f'E_{j+1}', lowBound=0, cat='Continuous') for j in range(num_strata)]

# Variáveis de Segundo Estágio (dependem do cenário)
y_mo = pulp.LpVariable.dicts("y_mo", range(num_scenarios), lowBound=0, cat='Continuous')
y_deficit = pulp.LpVariable.dicts("y_deficit", range(num_scenarios), lowBound=0, cat='Continuous')

# --- Função Objetivo ---
# [cite_start]Maximizar a receita esperada menos o custo de recurso esperado [cite: 327]
expected_revenue = pulp.lpSum(
    probabilities[s] * pulp.lpSum(scenarios[s]['c'][j] * E[j] for j in range(num_strata))
    for s in range(num_scenarios)
)

expected_recourse_cost = pulp.lpSum(
    probabilities[s] * (q_mo * y_mo[s] + q_deficit * y_deficit[s])
    for s in range(num_scenarios)
)

model += expected_revenue - expected_recourse_cost, "Lucro_Esperado_Liquido"

# --- Restrições ---

# Restrições de Primeiro Estágio
for j in range(num_strata):
    model += E[j] <= area_limits_A[j], f'Limite_Area_E{j+1}'

# Restrições de Segundo Estágio (para cada cenário)
for s in range(num_scenarios):
    # Restrição de Mão de Obra
    labor_needed = pulp.lpSum(base_labor_m[j] * E[j] for j in range(num_strata))
    model += labor_needed <= scenarios[s]['M_max'] + y_mo[s], f'Restricao_MO_Cenario_{s+1}'

    # Restrição de Demanda de Lenha
    firewood_produced = pulp.lpSum(scenarios[s]['r_lenha'][j] * E[j] for j in range(num_strata))
    model += firewood_produced >= firewood_demand_min - y_deficit[s], f'Restricao_Lenha_Cenario_{s+1}'

# --------------------------------------------------------------------------
# 4. RESOLUÇÃO E APRESENTAÇÃO DOS RESULTADOS
# --------------------------------------------------------------------------

# Resolve o problema
print("Resolvendo o modelo de otimização estocástica...")
model.solve()
print("Modelo resolvido.\n")

# Exibe os resultados
print(f"Status da Solução: {pulp.LpStatus[model.status]}\n")

print("--- Decisões Ótimas de Alocação de Área (1º Estágio) ---")
print("Esta é a estratégia robusta a ser implementada, considerando as incertezas:")
for j in range(num_strata):
    print(f"Área a ser explorada no Estrato E_{j+1}: {E[j].varValue:.4f} ha")

print("\n--- Valor Esperado da Função Objetivo ---")
print(f"Lucro Líquido Esperado: US$ {pulp.value(model.objective):.2f}\n")

print("--- Análise dos Custos de Recurso Esperados ---")
total_expected_mo_cost = sum(probabilities[s] * q_mo * y_mo[s].varValue for s in range(num_scenarios))
total_expected_deficit_cost = sum(probabilities[s] * q_deficit * y_deficit[s].varValue for s in range(num_scenarios))

print(f"Custo esperado com contratação de mão de obra extra: US$ {total_expected_mo_cost:.2f}")
print(f"Custo esperado com penalidades por déficit de lenha: US$ {total_expected_deficit_cost:.2f}")

Gerando 50 cenários de incerteza...
Cenários gerados com sucesso.

Resolvendo o modelo de otimização estocástica...
Modelo resolvido.

Status da Solução: Optimal

--- Decisões Ótimas de Alocação de Área (1º Estágio) ---
Esta é a estratégia robusta a ser implementada, considerando as incertezas:
Área a ser explorada no Estrato E_1: 21.6600 ha
Área a ser explorada no Estrato E_2: 14.1700 ha
Área a ser explorada no Estrato E_3: 10.6702 ha
Área a ser explorada no Estrato E_4: 0.0000 ha
Área a ser explorada no Estrato E_5: 3.5389 ha

--- Valor Esperado da Função Objetivo ---
Lucro Líquido Esperado: US$ 13142.29

--- Análise dos Custos de Recurso Esperados ---
Custo esperado com contratação de mão de obra extra: US$ 1874.11
Custo esperado com penalidades por déficit de lenha: US$ 314.73
